In [1]:
%load_ext autoreload
%autoreload 2

In [31]:
import pandas as pd
from import_casa import casa
import numpy as np

In [8]:
seeds = pd.read_excel(casa.get_data_path() / "ontology/CHT_terms.xlsx", sheet_name="CHTBD_seeds")

In [10]:
chtmenus = pd.read_excel(casa.get_data_path() / "ontology/CHT_terms.xlsx", sheet_name="website_menus")

In [38]:
mapping_sheet = pd.read_excel(casa.get_data_path() / "ontology/CHT_terms.xlsx", sheet_name="menus_seeds_mappings")

## Build mappings

In [101]:
from itertools import product
mappings = set()
for entry in mapping_sheet.dropna().itertuples():
    for seed, menu in product(
            str(entry.Seed).split(), 
            [entry.Tab]):        
        mappings.add((seed, menu))    
mapping_table = pd.DataFrame(mappings, columns=["Seed", "Tab"])

In [139]:
mapping_table.sort_values("Seed").to_csv(casa.get_data_path() / "ontology/mapping_pairs.csv")

In [136]:
mapping_table.sort_values("Tab")

,Seed,Tab
161,測速,CHT Wi-Fi
157,Speed,CHT Wi-Fi
155,收訊,CHT Wi-Fi
85,網速,CHT Wi-Fi
15,速度,CHT Wi-Fi
...,...,...
33,video,電影199
7,video,霹靂
36,預付卡,預付卡 上網/通話e儲值
159,儲值,預付卡 上網/通話e儲值


## Seeds as reference: Menus to Seeds

### print seeds hierarchy

In [128]:
fout = open(casa.get_data_path() / "ontology/seed_tree.txt", "w", encoding="UTF-8")
for category in seeds.Category.unique():
    fout.write(category + "\n")
    fout.write("----\n")
    fout.write(",".join(seeds.astype(str).loc[seeds.Category==category, "Seed"]))
    fout.write("\n\n")
fout.close()

In [104]:
seeds_mapped = seeds.merge(mapping_table, on="Seed", how='left')\
  .groupby(["Category", "Seed"])\
  .aggregate(
    nMapped=("Tab", lambda x: x.dropna().shape[0]),
    tabs=("Tab", lambda x: ",".join(x) if x.dropna().shape[0] > 0 else ""))\
  .reset_index()
seeds_mapped

,Category,Seed,nMapped,tabs
0,加值服務,FriDay,0,
1,加值服務,HAMI,8,"Hami 書城,序號兌換,功能介紹,Hami Point網站,贈點方案與兌兌Point,如何..."
2,加值服務,IOT,0,
3,加值服務,NFC,0,
4,加值服務,Pay,5,"線上繳費,更多帳單與發票相關,定期付款設定,我的帳單,申請電子帳單"
...,...,...,...,...
89,通訊品質,訊號,6,"漫遊上網方案,升級5G,航空漫遊,流量轉贈,精采5G方案,船舶漫遊"
90,通訊品質,速度,10,"CHT Wi-Fi,HiNet ADSL,HiNet 光世代,網路客服中心,CHT Wi-F..."
91,通訊品質,郊區,0,
92,通訊品質,頻段,0,


In [113]:
seeds_mapped.groupby("Category").aggregate(
    nSeeds=('Seed', lambda x: len(x)),
    covered_ratio=('nMapped', lambda x: (x>0).sum()/x.size))

,nSeeds,covered_ratio
Category,,
加值服務,33,0.484848
資費方案,34,0.705882
通訊品質,27,0.259259


## CHT Menus as reference: Seeds -> Menus

In [106]:
chtmenus

,Category,Subcategory,tab
0,行動上網通話,5G專區,升級5G
1,行動上網通話,5G專區,精采5G方案
2,行動上網通話,5G專區,應用服務
3,行動上網通話,5G專區,行動VIP
4,行動上網通話,5G專區,流量轉贈
...,...,...,...
123,服務與會員,中華電信VIP,行動VIP
124,服務與會員,中華電信VIP,HiNet VIP
125,線上客服,線上客服,行動業務
126,線上客服,線上客服,固網業務


### print tab hierarchy

In [130]:
edges = []
for category in chtmenus.Category.unique():
    edges.append(("CHT Website", category))
    for subcat in chtmenus.loc[chtmenus.Category==category, "Subcategory"].unique():
        edges.append((category, subcat))
        for seed in chtmenus.loc[chtmenus.Subcategory==subcat, "tab"]:
            edges.append((subcat, seed))            
pd.DataFrame(edges, columns=["src", "tgt"])\
    .to_excel(casa.get_data_path() / "ontology/menu_edges.xlsx", index=False)

In [135]:
chtmenus.groupby("Category").aggregate(
    n_sub=("Subcategory", lambda x: x.unique().size),
    n_tab=("tab", lambda x: len(x)))

,n_sub,n_tab
Category,,
MOD與智慧生活,5,20
家庭上網與通訊,6,26
服務與會員,6,26
線上客服,1,3
行動上網通話,9,53


In [107]:
tabs_mapped = chtmenus.merge(mapping_table, left_on="tab", right_on="Tab", how='left')\
  .groupby(["Category", "Subcategory", "tab"])\
  .aggregate(
    nMapped=("Seed", lambda x: x.dropna().shape[0]),
    seeds=("Seed", lambda x: ",".join(x) if x.dropna().shape[0] > 0 else ""))\
  .reset_index()
tabs_mapped

,Category,Subcategory,tab,nMapped,seeds
0,MOD與智慧生活,TV APP與更多服務,KKBOX,0,
1,MOD與智慧生活,TV APP與更多服務,KKTV,1,TV
2,MOD與智慧生活,TV APP與更多服務,Netflix,1,video
3,MOD與智慧生活,TV APP與更多服務,聲控助理,0,
4,MOD與智慧生活,TV APP與更多服務,親子199,0,
...,...,...,...,...,...
120,行動上網通話,預付卡與儲值,HoHo(卡)服務,1,預付卡
121,行動上網通話,預付卡與儲值,儲值紀錄查詢,2,"退費,儲值"
122,行動上網通話,預付卡與儲值,序號兌換,2,"HAMI,儲值"
123,行動上網通話,預付卡與儲值,親子預付卡,1,儲值


In [112]:
tabs_mapped.groupby("Category").aggregate(
    nsub=('Subcategory', lambda x: len(x)),
    covered_ratio=('nMapped', lambda x: (x>0).sum()/x.size))

,nsub,covered_ratio
Category,,
MOD與智慧生活,20,0.600000
家庭上網與通訊,26,0.384615
服務與會員,26,0.615385
線上客服,3,0.000000
行動上網通話,50,0.600000


In [111]:
with pd.ExcelWriter(casa.get_data_path() / "ontology/mappings.xlsx") as xlsx:
    seeds_mapped.to_excel(xlsx, sheet_name="seeds_mapped", index=None)
    tabs_mapped.to_excel(xlsx, sheet_name="tabs_mapped", index=None)